<a href="https://colab.research.google.com/github/Felix-Leonel/Codigos-Hate-Speech/blob/main/BERT_Hate_Speech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install transformers
!pip install gdown
!pip install tokenizers
!pip install captum
!pip install wget

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 46.1 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-m

In [ ]:
import torch
import logging
import os
import pandas as pd
import numpy as np
import time
import datetime
import random
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tokenizers import (ByteLevelBPETokenizer,
                            SentencePieceBPETokenizer,
                            BertWordPieceTokenizer)
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AutoTokenizer, AutoModelForMaskedLM
from transformers import BertForSequenceClassification, AdamW, BertConfig, BertModel
from transformers import BertModel, BertForMaskedLM, PreTrainedTokenizer
from tokenizers import BertWordPieceTokenizer
from transformers import get_linear_schedule_with_warmup
from transformers import AutoTokenizer
from datasets import load_dataset
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
import wget
import gdown
from sklearn.preprocessing import LabelEncoder

In [ ]:
!git lfs install
!git clone https://huggingface.co/neuralmind/bert-base-portuguese-cased

Git LFS initialized.
Cloning into 'bert-base-portuguese-cased'...
remote: Enumerating objects: 39, done.
remote: Total 39 (delta 0), reused 0 (delta 0), pack-reused 39
Unpacking objects: 100% (39/39), 103.13 KiB | 1.69 MiB/s, done.
Filtering content: 100% (3/3), 1.30 GiB | 9.40 MiB/s, done.


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Python/OFFCOMBR_CSV/OffComBR2.csv')
for i in df.index:
  if(df['@@class'].loc[i] == 'yes'):
    df['@@class'].loc[i] = 1
  if(df['@@class'].loc[i] == 'no'):
    df['@@class'].loc[i] = 0
df.rename(columns={'document': 'text'}, inplace = True)
df.rename(columns={'@@class': 'label'}, inplace = True)

df

In [ ]:
if torch.cuda.is_available():

    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

logging.basicConfig(level=logging.INFO)


df['text'] = df['text'].replace('#\w*', '', regex=True)
df['text'] = df['text'].replace('RT @\w*: ', '', regex=True)
df['text'] = df['text'].replace('@\w* ', '', regex=True)

df.head(5)

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


,id,label,text
0,1,1,Votaram no PEZAO Agora tomem no CZAO
1,2,0,cuidado com a poupanca pessoal Lembram o que a...
2,3,0,Sabe o que eu acho engracado os nossos governa...
3,4,1,os cariocas tem o que merecem um pessoal que s...
4,5,0,Podiam retirar dos lucros dos bancos


In [ ]:
sentencas = df['text'].values
labels = df['label'].values
encoder = LabelEncoder()
labels = encoder.fit_transform(labels)
labels = labels.astype(np.int64)
print(type(sentencas))

<class 'numpy.ndarray'>


In [ ]:
tokenizer = BertWordPieceTokenizer('/content/bert-base-portuguese-cased/vocab.txt', lowercase=False, strip_accents=False)
print(tokenizer)

Tokenizer(vocabulary_size=29794, model=BertWordPiece, unk_token=[UNK], sep_token=[SEP], cls_token=[CLS], pad_token=[PAD], mask_token=[MASK], clean_text=True, handle_chinese_chars=True, strip_accents=False, lowercase=False, wordpieces_prefix=##)


In [ ]:
tokenizer.enable_truncation(max_length=512)
tokenizer.enable_padding()
output = tokenizer.encode_batch(sentencas.tolist())
ids = [x.ids for x in output]
attention_mask = [x.attention_mask for x in output]

In [ ]:
print(len(attention_mask))

1250


In [ ]:
print(output[0])

Encoding(num_tokens=136, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])


In [ ]:
print(output[0].tokens)

['[CLS]', 'Vo', '##taram', 'no', 'P', '##E', '##Z', '##A', '##O', 'Agora', 'tom', '##em', 'no', 'C', '##Z', '##A', '##O', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD

In [ ]:
train_input_ids, validation_input_ids, train_labels, validation_labels = train_test_split(ids, labels, random_state=2018, test_size=0.2)
train_attention_mask, validation_attention_mask, _, _ = train_test_split(attention_mask, labels, random_state=2018, test_size=0.2)

In [ ]:
print(train_input_ids[0])

[101, 8484, 368, 2541, 333, 5139, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
print(train_attention_mask[0])

[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
train_input_ids, validation_input_ids, train_labels, validation_labels = train_test_split(ids, labels, random_state=2018, test_size=0.2)
train_attention_mask, validation_attention_mask, _, _ = train_test_split(attention_mask, labels, random_state=2018, test_size=0.2)

train_input_tensor = torch.tensor(train_input_ids, dtype=torch.long)
validation_input_tensor = torch.tensor(validation_input_ids, dtype=torch.long)

train_labels_tensor = torch.tensor(train_labels, dtype=torch.long)
validation_labels_tensor = torch.tensor(validation_labels, dtype=torch.long)

train_mask_tensor = torch.tensor(train_attention_mask, dtype=torch.long)
validation_mask_tensor = torch.tensor(validation_attention_mask, dtype=torch.long)


In [ ]:
batch_size = 8

train_data = TensorDataset(train_input_tensor, train_mask_tensor, train_labels_tensor)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_input_tensor, validation_mask_tensor, validation_labels_tensor)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


In [ ]:
model = BertForSequenceClassification.from_pretrained(
    'neuralmind/bert-base-portuguese-cased',
    num_labels = 2,
    output_attentions = True,
    output_hidden_states = True,
)

model.cuda()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(29794, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
params = list(model.named_parameters())
print('O modelo Bert tem {:} parâmetros com nomes diferentes.\n'.format(len(params)))

O modelo Bert tem 201 parâmetros com nomes diferentes.



In [ ]:
print('==== Embedding Layer ====\n')
for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (29794, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)


In [ ]:
print('\n==== Primeiro Transformer ====\n')
for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))


==== Primeiro Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (768,)
bert.encoder.layer.0.attention.output.LayerNorm.weight        (768,)
bert.encoder.layer.0.attention.output.LayerNorm.bias          (768,)
bert.encoder.layer.0.intermediate.dense.weight           (3072, 768)
bert.encoder.layer.0.intermediate.dense.bias                 (3072,)
bert.encoder.layer.0.output.dense.weight                 (768, 3072)
bert.encoder.layer.0.output.dense.bias                        (768,)
b

In [ ]:
print('\n==== Output Layer ====\n')
for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))


==== Output Layer ====

bert.pooler.dense.weight                                  (768, 768)
bert.pooler.dense.bias                                        (768,)
classifier.weight                                           (2, 768)
classifier.bias                                                 (2,)


In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = 1e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
epochs = 3
total_steps = epochs * len(train_dataloader)
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [ ]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

loss_values = []

for epoch_i in range(0, epochs):
    print("")
    print('======== Época {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Treinamento...')

    t0 = time.time()
    total_loss = 0

    model.train()
    device = torch.device("cuda")

    for step, batch in enumerate(train_dataloader):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()

        outputs = model(b_input_ids,
                    token_type_ids=None, #USADO QUANDO É NEXT SEQUENCE
                    attention_mask=b_input_mask,
                    labels=b_labels)

        loss = outputs[0]
        hidden_state = outputs[2]

        total_loss += loss.item()

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()

        scheduler.step()

    avg_train_loss = total_loss / len(train_dataloader)

    loss_values.append(avg_train_loss)

    print("Running Validation...")
    t0 = time.time()

    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    for batch in validation_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        with torch.no_grad():
          outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

        logits = outputs[0]

        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        tmp_eval_accuracy = flat_accuracy(logits, label_ids)

        eval_accuracy += tmp_eval_accuracy

        nb_eval_steps +=1

    print("  Acurácia: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Tempo de Validação: {:}".format(format_time(time.time() - t0)))

print("FIM DO TREINAMENTO")


======== Época 1 / 3 ========
Treinamento...
Running Validation...
  Acurácia: 0.82
  Tempo de Validação: 0:00:02

======== Época 2 / 3 ========
Treinamento...
Running Validation...
  Acurácia: 0.83
  Tempo de Validação: 0:00:02

======== Época 3 / 3 ========
Treinamento...
Running Validation...
  Acurácia: 0.84
  Tempo de Validação: 0:00:02
FIM DO TREINAMENTO


In [ ]:
def Validar_Modelo(prediction_dataloader, batch_size):
  resultado_predicoes = []
  resultados_esperados = []
  falsos_positivos = []
  falsos_negativos = []
  verdadeiros_positivos = []
  verdadeiros_negativos =[]
  nb_eval_steps = 0
  eval_accuracy = 0
  tmp_eval_accuracy = 0
  for batch in prediction_dataloader:
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)

    with torch.no_grad():
      outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

    logits = outputs[0]
    attention=outputs[-1]
    all_hidden_state = outputs[-2]

    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    reconstruct_input_id = b_input_ids.to('cpu').numpy()

    for logit, label, inputs, att in zip(logits, label_ids, reconstruct_input_id, attention):
      resultado_predicoes.append(np.argmax(logit))
      resultados_esperados.append(label)

      if (label != np.argmax(logit)):
        if (label == 1 and np.argmax(logit) == 0):
          falsos_negativos.append(inputs)
        elif(label == 0 and np.argmax(logit) == 1):
          falsos_positivos.append(inputs)
      else:
        if (label == 1 and np.argmax(logit) == 1):
          verdadeiros_positivos.append(inputs)
        elif (label == 0 and np.argmax(logit) == 0):
          verdadeiros_negativos.append(inputs)

    tmp_eval_accuracy = flat_accuracy(logits, label_ids)

    eval_accuracy += tmp_eval_accuracy

    nb_eval_steps +=1

  print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
  print("Accuracia: ", accuracy_score(resultados_esperados, resultado_predicoes))
  print("Balanced Accuracy:", balanced_accuracy_score(resultados_esperados, resultado_predicoes))
  print("F1 Score:", f1_score(resultados_esperados, resultado_predicoes, average='weighted'))
  print("Recall:", recall_score(resultados_esperados, resultado_predicoes, average='weighted'))
  print("Precision: ", precision_score(resultados_esperados, resultado_predicoes, average='weighted'))
  tn, fp, fn, tp = confusion_matrix(resultados_esperados, resultado_predicoes).ravel()
  print ("True Negative: ", tn)
  print ("False Positive: ", fp)
  print ("False Negative: ", fn)
  print ("True Positive: ", tp)

In [ ]:
df['text'] = df['text'].replace('#\w*', '', regex=True)
df['text'] = df['text'].replace('RT @\w*: ', '', regex=True)
df['text'] = df['text'].replace('@\w* ', '', regex=True)
df['text'] = df['text'].str.replace(r"[^\w\s]", "", regex=True)
df['text'] = df['text'].str.lower().str.strip()

sentencas = df['text'].values
labels = df['label'].values
labels = labels.astype(np.int64)

tokenizer.enable_truncation(max_length=100)
tokenizer.enable_padding()

output = tokenizer.encode_batch(sentencas.tolist())


ids=[x.ids for x in output]
tokens = [x.tokens for x in output]
attention_mask = [x.attention_mask for x in output]

prediction_input = torch.tensor(ids)
prediction_mask = torch.tensor(attention_mask)
prediction_labels = torch.tensor(labels)


In [ ]:
batch_size = 8

prediction_data = TensorDataset(prediction_input, prediction_mask, prediction_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

Validar_Modelo(prediction_dataloader, batch_size)

Accuracy: 0.88
Accuracia:  0.8856
Balanced Accuracy: 0.8760945923047541
F1 Score: 0.8860849144880278
Recall: 0.8856
Precision:  0.8868431553266468
True Negative:  752
False Positive:  79
False Negative:  64
True Positive:  355
